# NFL DFS Optimizer
#### Developed by Dan McDonough
#### Updated April 14, 2023

The purpose of this notebook is to help construct an optimal DFS lineup for the week.

I don't have a whole lot of NFL domain knowledge, so I didn't want to spend time trying to build a player projection model. Instead, I wanted to try to build competitive advantage in my league by being able to maximize the number of projected points drafted, with a given set of projections taken as ground truth.

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import timeit
import numpy as np
from pulp import *

from get_data import scrape_qb, scrape_rb, scrape_wr, scrape_te, scrape_dst, clean_dst, combine_projections,\
                        clean_name, get_salaries, check_players
from apply_scoring import get_scoring_dict, qb_scoring, rb_scoring, wr_scoring, te_scoring, dst_scoring

### 1. Scrape Projections

First, I need to scrape a source of fantasy projections. For my purposes, I am going to treat these projections as truth, and determine how to optimally select a lineup that maximizes the number of projected fantasy points.

In [2]:
week = 11
droplist = ['TEN','GB','KC','LAC','SF','ARI','JAC','MIA','SEA','TB']

#### QB Projections

In [3]:
qb_projections = scrape_qb(week)
qb_projections.head()

,player_stat,team,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,Marcus Mariota,PHI,24.4,15.6,187.5,1.3,0.5,5.7,34.6,0.2,0.2
1,Russell Wilson,DEN,33.4,20.4,241.7,1.3,0.7,4.0,16.0,0.1,0.2
2,Jimmy Garoppolo,LV,31.6,21.2,256.1,1.5,0.8,1.8,3.9,0.1,0.2
3,Taylor Heinicke,ATL,32.5,20.6,229.0,1.2,0.9,3.7,14.0,0.1,0.2
4,Jacoby Brissett,WAS,32.6,20.6,222.3,1.1,0.6,4.7,18.9,0.1,0.2


Next, I will add scoring for Draft Kings

In [4]:
scoring_dict = get_scoring_dict()
qb_projections['points'] = qb_projections.apply(qb_scoring, args =(scoring_dict, ), axis=1)
qb_projections.head(1)

,player_stat,team,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Marcus Mariota,PHI,24.4,15.6,187.5,1.3,0.5,5.7,34.6,0.2,0.2,16.66


#### RB

In [5]:
rb_projections = scrape_rb(week,80)
rb_projections.head()

,player_stat,team,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
0,Saquon Barkley,NYG,22.2,101.6,0.8,3.0,24.4,0.1,0.1
1,Derrick Henry,TEN,21.2,98.0,0.8,1.8,13.9,0.0,0.1
2,Christian McCaffrey,SF,14.2,63.6,0.6,4.8,40.8,0.2,0.1
3,Austin Ekeler,LAC,11.7,52.4,0.6,6.2,46.9,0.3,0.1
4,Joe Mixon,CIN,17.1,68.3,0.6,3.8,29.8,0.2,0.1


In [6]:
rb_projections['points'] = rb_projections.apply(rb_scoring, args= (scoring_dict,), axis=1)
rb_projections.head(1)

,player_stat,team,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles,points
0,Saquon Barkley,NYG,22.2,101.6,0.8,3.0,24.4,0.1,0.1,23.9


#### WR

In [7]:
wr_projections = scrape_wr(week)
wr_projections.head()

,player_stat,team,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
0,Justin Jefferson,MIN,6.9,100.0,0.6,0.2,1.4,0.0,0.1
1,Stefon Diggs,BUF,7.3,94.9,0.7,0.0,0.1,0.0,0.0
2,Davante Adams,LV,6.9,85.7,0.6,0.3,1.1,0.0,0.0
3,CeeDee Lamb,DAL,5.9,81.7,0.6,0.5,2.8,0.0,0.0
4,A.J. Brown,PHI,5.5,81.2,0.6,0.0,0.2,0.0,0.0


In [8]:
wr_projections['points'] = wr_projections.apply(wr_scoring, args= (scoring_dict,), axis=1)
wr_projections.head(1)

,player_stat,team,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Justin Jefferson,MIN,6.9,100.0,0.6,0.2,1.4,0.0,0.1,23.54


#### TE

In [9]:
te_projections = scrape_te(week)
te_projections.head()

,player_stat,team,receptions,receive_yards,receive_tds,fumbles
0,Travis Kelce,KC,6.7,80.9,0.8,0.0
1,Mark Andrews,BAL,4.9,59.9,0.6,0.0
2,T.J. Hockenson,MIN,4.8,53.0,0.5,0.0
3,George Kittle,SF,4.3,53.0,0.5,0.0
4,Pat Freiermuth,PIT,4.4,48.9,0.4,0.0


In [10]:
te_projections['points'] = te_projections.apply(te_scoring, args= (scoring_dict,), axis=1)
te_projections.head(1)

,player_stat,team,receptions,receive_yards,receive_tds,fumbles,points
0,Travis Kelce,KC,6.7,80.9,0.8,0.0,19.59


#### D/ST

In [11]:
dst_projections = scrape_dst(week)
dst_projections.head()

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against
0,Philadelphia Eagles,2.9,1.1,0.7,0.2,0.0,17.9,318.6
1,New England Patriots,3.0,1.1,0.6,0.2,0.0,16.6,295.8
2,Cincinnati Bengals,2.4,1.3,0.6,0.2,0.0,18.8,323.3
3,New Orleans Saints,2.6,1.0,0.6,0.2,0.0,19.0,312.9
4,Buffalo Bills,2.5,0.9,0.7,0.2,0.0,18.8,346.0


In [12]:
dst_projections = clean_dst(dst_projections)
dst_projections.head(1)

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against,team
0,Eagles,2.9,1.1,0.7,0.2,0.0,17.9,318.6,PHI


In [13]:
dst_projections['points'] = dst_projections.apply(dst_scoring, args= (scoring_dict,), axis=1)
dst_projections.head(1)

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against,team,points
0,Eagles,2.9,1.1,0.7,0.2,0.0,17.9,318.6,PHI,9.585714


#### Combine projections

In [14]:
all_projections = combine_projections(qb_projections,rb_projections,wr_projections,te_projections,dst_projections)
all_projections.head()

,player_stat,team,points
0,Saquon Barkley,NYG,23.90
0,Justin Jefferson,MIN,23.54
3,Austin Ekeler,LAC,21.43
1,Stefon Diggs,BUF,21.00
2,Christian McCaffrey,SF,19.94


### 2. Load salaries

Now I will load the player salaries from Draft Kings.

In [15]:
df_raw = get_salaries()
df_raw.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,WR,Justin Jefferson (25323474),Justin Jefferson,25323474,WR/FLEX,9100,DAL@MIN 11/20/2022 04:25PM ET,MIN,25.25
1,WR,Cooper Kupp (25323476),Cooper Kupp,25323476,WR/FLEX,8900,LAR@NO 11/20/2022 01:00PM ET,LAR,24.16
2,RB,Saquon Barkley (25323208),Saquon Barkley,25323208,RB/FLEX,8900,DET@NYG 11/20/2022 01:00PM ET,NYG,21.31
3,WR,Davante Adams (25323478),Davante Adams,25323478,WR/FLEX,8700,LV@DEN 11/20/2022 04:05PM ET,LV,22.03
4,QB,Josh Allen (25323137),Josh Allen,25323137,QB,8500,CLE@BUF 11/20/2022 01:00PM ET,BUF,29.21


In [16]:
all_projections.shape

(330, 3)

In [17]:
all_projections = all_projections[all_projections['team'].apply(lambda x: True if x not in droplist else False)].copy()
all_projections.shape

(268, 3)

First, I need to standardize names across data sets

In [18]:
check_players(te_projections, 'T', droplist, df_raw)

In projections, but not in salaries: 
13   Robert Tonyan
18   Austin Hooper


In [19]:
rb_projections[rb_projections['player_stat']=='Amari Rodgers']

,player_stat,team,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles,points


In [20]:
df_raw[(df_raw['TeamAbbrev']=='HOU')&(df_raw['Position']=='RB')]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
26,RB,Dameon Pierce (25323228),Dameon Pierce,25323228,RB/FLEX,6500,WAS@HOU 11/20/2022 01:00PM ET,HOU,15.64
146,RB,Rex Burkhead (25323310),Rex Burkhead,25323310,RB/FLEX,4400,WAS@HOU 11/20/2022 01:00PM ET,HOU,6.36
202,RB,Dare Ogunbowale (25323418),Dare Ogunbowale,25323418,RB/FLEX,4000,WAS@HOU 11/20/2022 01:00PM ET,HOU,1.61
203,RB,Darius Anderson (25323420),Darius Anderson,25323420,RB/FLEX,4000,WAS@HOU 11/20/2022 01:00PM ET,HOU,0.00
204,RB,Royce Freeman (25323422),Royce Freeman,25323422,RB/FLEX,4000,WAS@HOU 11/20/2022 01:00PM ET,HOU,0.00
205,RB,Troy Hairston (25323424),Troy Hairston,25323424,RB/FLEX,4000,WAS@HOU 11/20/2022 01:00PM ET,HOU,0.47
206,RB,Gerrid Doaks (25323426),Gerrid Doaks,25323426,RB/FLEX,4000,WAS@HOU 11/20/2022 01:00PM ET,HOU,0.00


In [21]:
# all_projections['player_stat'].replace('Stanley Berryhill III','Stanley Berryhill',inplace=True)
# all_projections['player_stat'].replace('Patrick Mahomes II','Patrick Mahomes',inplace=True)
# all_projections['player_stat'].replace('PJ Walker','P.J. Walker',inplace=True)
# all_projections['player_stat'].replace('Ken Walker III','Kenneth Walker III',inplace=True)
# all_projections['player_stat'].replace('Troy Hairston II','Troy Hairston',inplace=True)
all_projections['player_stat'].replace('Demetric Felton Jr.','Demetric Felton',inplace=True)
# all_projections['player_stat'].replace("D'Vonte Price","D'vonte Price",inplace=True)
# all_projections['player_stat'].replace('Patrick Taylor Jr.','Patrick Taylor',inplace=True)
all_projections['player_stat'].replace('D.J. Chark Jr.','DJ Chark Jr.',inplace=True)
# all_projections['player_stat'].replace('Ray-Ray McCloud','Ray-Ray McCloud III',inplace=True)
all_projections['player_stat'].replace('Keelan Cole Sr.','Keelan Cole',inplace=True)
# all_projections['player_stat'].replace('K.J. Hamler','KJ Hamler',inplace=True)
# all_projections['player_stat'].replace('Chigoziem Okonkwo','Chig Okonkwo',inplace=True)
# all_projections['player_stat'].replace('Elijah Mitchell','Eli Mitchell',inplace=True)
# all_projections['player_stat'].replace('D.K. Metcalf','DK Metcalf',inplace=True)
# all_projections['player_stat'].replace('D.J. Moore','DJ Moore',inplace=True)
# all_projections['player_stat'].replace('Darrell Henderson','Darrell Henderson Jr.',inplace=True)
# all_projections['player_stat'].replace("D'Wayne Eskridge",'Dee Eskridge',inplace=True)
# all_projections['player_stat'].replace('Van Jefferson','Van Jefferson Jr.',inplace=True)
# all_projections['player_stat'].replace('Josh Palmer','Joshua Palmer',inplace=True)
# all_projections['player_stat'].replace('Chris Herndon IV','Chris Herndon',inplace=True)
# all_projections['player_stat'].replace('Jakeem Grant Sr.','Jakeem Grant',inplace=True)
# all_projections['player_stat'].replace('Ray-Ray McCloud','Ray-Ray McCloud III',inplace=True)
# all_projections['player_stat'].replace('Phillip Walker','P.J. Walker',inplace=True)
# all_projections['player_stat'].replace('J.J. Arcega-Whiteside','JJ Arcega-Whiteside',inplace=True)
# all_projections['player_stat'].replace('James Proche II','James Proche',inplace=True)
# all_projections['player_stat'].replace('Stanley Morgan Jr.','Stanley Morgan',inplace=True)
# all_projections['player_stat'].replace('K.J. Hill Jr.','K.J. Hill',inplace=True)
# all_projections['player_stat'].replace('Victor Bolden Jr.','Victor Bolden',inplace=True)
# all_projections['player_stat'].replace('Alex Armah Jr.','Alex Armah',inplace=True)
# all_projections['player_stat'].replace('John Ross','John Ross III',inplace=True)
# all_projections['player_stat'].replace('Elijhaa Penny','Eli Penny',inplace=True)
# all_projections['player_stat'].replace('Adrian Killins Jr.','Adrian Killins',inplace=True)

In [22]:
combined_df = df_raw.merge(all_projections,how='left',left_on='Name',right_on='player_stat')
combined_df['points'] = combined_df['points'].fillna(0)

provide option to drop specific players who are listed as questionable

In [23]:
# combined_df = combined_df[combined_df['Name'].apply(lambda x: True if x not in ['Tyler Higbee','Jakobi Meyers'] else False)].copy()

### 3. Build optimizer

First, I will create the necessary data structures

In [24]:
prob = LpProblem("combined_dfs",LpMaximize)

players = list(combined_df['ID'])
salaries = dict(zip(players,combined_df['Salary']))
points = dict(zip(players,combined_df['points']))
qbs = dict(zip(players,(combined_df['Position']=='QB')*1))
rbs = dict(zip(players,(combined_df['Position']=='RB')*1))
wrs = dict(zip(players,(combined_df['Position']=='WR')*1))
tes = dict(zip(players,(combined_df['Position']=='TE')*1))
defs = dict(zip(players,(combined_df['Position']=='DST')*1))
player_count = dict(zip(players,[1]*len(combined_df)))

player_vars = LpVariable.dicts("Players",players,lowBound=0,upBound=1,cat='Integer')

Next, I will define the objective and constraints

In [25]:
prob += lpSum([points[i]*player_vars[i] for i in players])

prob += lpSum([salaries[f] * player_vars[f] for f in players]) <= 50000

prob += lpSum([qbs[f] * player_vars[f] for f in players]) == 1 
prob += lpSum([rbs[f] * player_vars[f] for f in players]) >= 2
prob += lpSum([rbs[f] * player_vars[f] for f in players]) <= 3
prob += lpSum([wrs[f] * player_vars[f] for f in players]) >= 3
prob += lpSum([wrs[f] * player_vars[f] for f in players]) <= 4
prob += lpSum([tes[f] * player_vars[f] for f in players]) >= 1
prob += lpSum([tes[f] * player_vars[f] for f in players]) <= 2
prob += lpSum([defs[f] * player_vars[f] for f in players]) == 1
prob += lpSum([player_count[f] * player_vars[f] for f in players]) == 9

Solve problem

In [26]:
prob.solve()

1

In [30]:
players_out = []

for v in prob.variables():
    if v.varValue>0:
        players_out.append(int('-'.join(str(v).split('_')[1:])))
        
combined_df['keep'] = combined_df['ID'].apply(lambda x: 1 if x in players_out else 0)
combined_df['value'] = combined_df['points']/combined_df['Salary']*1000

In [31]:
combined_df[combined_df['keep']==1]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,player_stat,team,points,keep,value
0,WR,Justin Jefferson (25323474),Justin Jefferson,25323474,WR/FLEX,9100,DAL@MIN 11/20/2022 04:25PM ET,MIN,25.25,Justin Jefferson,MIN,23.540000,1,2.586813
2,RB,Saquon Barkley (25323208),Saquon Barkley,25323208,RB/FLEX,8900,DET@NYG 11/20/2022 01:00PM ET,NYG,21.31,Saquon Barkley,NYG,23.900000,1,2.685393
26,RB,Dameon Pierce (25323228),Dameon Pierce,25323228,RB/FLEX,6500,WAS@HOU 11/20/2022 01:00PM ET,HOU,15.64,Dameon Pierce,HOU,16.490000,1,2.536923
57,QB,Marcus Mariota (25323149),Marcus Mariota,25323149,QB,5500,CHI@ATL 11/20/2022 01:00PM ET,ATL,16.46,Marcus Mariota,PHI,16.660000,1,3.029091
115,WR,Garrett Wilson (25323538),Garrett Wilson,25323538,WR/FLEX,4900,NYJ@NE 11/20/2022 01:00PM ET,NYJ,12.51,Garrett Wilson,NYJ,13.230000,1,2.700000
152,WR,Parris Campbell (25323564),Parris Campbell,25323564,WR/FLEX,4300,PHI@IND 11/20/2022 01:00PM ET,IND,9.72,Parris Campbell,NYG,10.200000,1,2.372093
161,TE,Pat Freiermuth (25323916),Pat Freiermuth,25323916,TE/FLEX,4200,CIN@PIT 11/20/2022 04:25PM ET,PIT,10.29,Pat Freiermuth,PIT,11.690000,1,2.783333
230,TE,Tyler Higbee (25323920),Tyler Higbee,25323920,TE/FLEX,4000,LAR@NO 11/20/2022 01:00PM ET,LAR,9.17,Tyler Higbee,LAR,11.610000,1,2.902500
560,DST,Texans (25324164),Texans,25324164,DST,2400,WAS@HOU 11/20/2022 01:00PM ET,HOU,5.67,Texans,HOU,7.728571,1,3.220238


In [32]:
combined_df[combined_df['keep']==1]['Salary'].sum()

49800

In [33]:
combined_df[combined_df['keep']==1]['points'].sum()

135.04857142857145